In [7]:
import torch
import torchvision
import cv2
import numpy as np
!git clone https://github.com/abewley/sort.git
from sort.sort import *

fatal: destination path 'sort' already exists and is not an empty directory.


In [8]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/gandalf/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-22 Python-3.8.10 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [9]:
object_tracker = Sort()
bounding_boxes_ids = np.array([])

In [10]:
# Opening the video file for reading
vidcap = cv2.VideoCapture('./video1.mp4')

# Initializing variables for processing frames
success = True
count2 = 0
img_array = []
track_ids = []
count = 0 

In [11]:
# Looping through each image of the video file
while success:
    # Reading the next image from the video file
    success,image = vidcap.read()

    # Checking if the image was successfully read
    if(success == 0 or image is None): 
        break

    # Perform object detection on the image
    pred = model(image)
    car_pred = pred.pred[0][pred.pred[0][:, 5] == 2]
    boxes = car_pred[:, :4].cpu().numpy()
    confidences = car_pred[:, 4].cpu().numpy()
    class_label = car_pred[:, 5].cpu().numpy()

    # Filter out low-confidence detections and track objects using an object tracker
    if bounding_boxes := [
        boxes[i] for i in range(len(class_label)) if (confidences[i] > 0.5)
    ]:
        bounding_boxes = np.array(bounding_boxes)
        tracks = object_tracker.update(bounding_boxes)

        # Update the list of bounding box IDs
        for i in range(tracks.shape[0]):
            bounding_boxes_ids = np.append(bounding_boxes_ids, np.array([tracks[i][4]]))

        # Count the number of unique bounding box IDs to get the car count for the frame
        count = np.unique(bounding_boxes_ids).shape[0]

        # Draw bounding boxes and labels around each car
        for i in range(bounding_boxes.shape[0]):
            cv2.rectangle(image, (int(bounding_boxes[i][0]), int(bounding_boxes[i][1])), (int(bounding_boxes[i][2]), int(bounding_boxes[i][3])), (0, 255, 0), 3)
            cv2.putText(image, "Car", (int(bounding_boxes[i][0])-10, int(bounding_boxes[i][1])-10),  cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the car count on the image
    cv2.putText(image, f'Number of Cars: {int(count)}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255,255,255), 3)

    # Add the current frame to an array of frames
    img_array.append(image)

    # If the current frame is one of the specified frames, save it as a JPEG file
    if(count2 == 12 or count2 == 490 or count2 == 200 or count2 == 400):
        cv2.imwrite("frame%d.jpg" % count2, img_array[-1])     # save frame as JPEG file

    # Increment the frame counter
    count2 += 1

In [6]:
# Determining the size of the frames and initializing a VideoWriter object for writing the output video file
height, width, layers = img_array[0].shape
size = (width,height)
out = cv2.VideoWriter('project_YOLO.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
# Writing each frame of the output video file
for i in range(len(img_array)):
    out.write(img_array[i])
    
# Releasing the VideoWriter object
out.release()